In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!unzip /content/gdrive/MyDrive/train.zip

Archive:  /content/gdrive/MyDrive/train.zip
  inflating: train/id_004wknd7qd.jpg  
  inflating: train/id_005sitfgr2.jpg  
  inflating: train/id_00stp9t6m6.jpg  
  inflating: train/id_012zxewnhx.jpg  
  inflating: train/id_0186qwq2at.jpg  
  inflating: train/id_01vb9vty05.jpg  
  inflating: train/id_01z6i8am9b.jpg  
  inflating: train/id_02lw7skww5.jpg  
  inflating: train/id_02olqz4tf5.jpg  
  inflating: train/id_03878to61t.jpg  
  inflating: train/id_04alcsykf1.jpg  
  inflating: train/id_04lhl9cypv.jpg  
  inflating: train/id_05udbcy9yj.jpg  
  inflating: train/id_06lh8r7g0p.jpg  
  inflating: train/id_06nrnxmqfw.jpg  
  inflating: train/id_06ya2x9n2i.jpg  
  inflating: train/id_08of6gx23n.jpg  
  inflating: train/id_08rn3cmwlk.jpg  
  inflating: train/id_09sjnce7rh.jpg  
  inflating: train/id_0b3wfufkrn.jpg  
  inflating: train/id_0b9myv7d84.jpg  
  inflating: train/id_0brsi9wkxu.jpg  
  inflating: train/id_0c29p9v7sv.jpg  
  inflating: train/id_0ch8257gec.jpg  
  inflating: train/i

In [1]:
import pandas as pd
train_df = pd.read_csv("Train.csv")
train_df.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_004wknd7qd_rgn.jpg,blast
2,id_005sitfgr2.jpg,brown
3,id_005sitfgr2_rgn.jpg,brown
4,id_00stp9t6m6.jpg,blast


In [2]:
train_df.drop(axis = 0, index = [i for i in range(1,train_df.shape[0],2)], inplace = True)
train_df = train_df.reset_index()
train_df.drop('index',axis = 1, inplace = True)
train_df.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_005sitfgr2.jpg,brown
2,id_00stp9t6m6.jpg,blast
3,id_012zxewnhx.jpg,blast
4,id_0186qwq2at.jpg,healthy


In [3]:
train_df['Label'].value_counts()

blast      1494
brown       766
healthy     410
Name: Label, dtype: int64

In [4]:
train_df['Label_int'] = train_df['Label'].replace(['blast','brown','healthy'], [0,1,2])
train_df

,Image_id,Label,Label_int
0,id_004wknd7qd.jpg,blast,0
1,id_005sitfgr2.jpg,brown,1
2,id_00stp9t6m6.jpg,blast,0
3,id_012zxewnhx.jpg,blast,0
4,id_0186qwq2at.jpg,healthy,2
...,...,...,...
2665,id_zydzdp046u.jpg,healthy,2
2666,id_zyoowbqcm3.jpg,healthy,2
2667,id_zz6gzk7p97.jpg,brown,1
2668,id_zz8ca2p67e.jpg,blast,0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_df['Image_id'],train_df['Label_int'], test_size=0.3, random_state=1234)


In [6]:
from torchvision import transforms

train_transform = transforms.Compose([
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=(0,180)),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])

    ])

val_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [7]:
import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
import cv2
from PIL import Image

class Rice_Dataset(Dataset):

    def __init__(self, images, labels, img_dir,transform):

        self.labels = labels
        self.img_dir = img_dir
        self.images = images
        self.transform = transform

    def __len__(self):

        return len(self.images)

    def __getitem__(self,idx):

        img_path = os.path.join(self.img_dir, self.images[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = self.transform(img)
        label = torch.tensor(self.labels[idx])

        return img, label

In [8]:
train_dataset = Rice_Dataset(X_train.values, y_train.values, "/content/train/",transform = train_transform)
valid_dataset = Rice_Dataset(X_val.values, y_val.values, "/content/train/",transform = val_transform)

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=150, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size = 150, shuffle = False)

In [10]:
import matplotlib.pyplot as plt
import torchvision
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

# fig,axs = plt.subplots(3,3)
# plt.subplots_adjust(wspace=0.5, hspace=0.4)
# for i in range(3):

#     for j in range(3):

#         img = train_features[i + j + i*2].squeeze()
#         label = train_labels[i + j + i*2 ]
#         if label == 0:

#             label = "blast"

#         elif label == 1:

#             label = "brown"

#         else:

#             label = "healthy"

#         axs[i,j].imshow(img);
#         axs[i,j].set_title(f"{label}");
#         axs[i,j].axis('off')


Feature batch shape: torch.Size([150, 3, 224, 224])
Labels batch shape: torch.Size([150])


**Building The Model**

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class Attn(nn.Module):
    def __init__(self, in_features,up_factor):

        super(Attn, self).__init__()
        self.up_factor = up_factor
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1, kernel_size=1, padding=0, bias=False)

    def forward(self, l, g):
        N, C, W, H = l.size()
        g = F.interpolate(g, scale_factor = self.up_factor, mode = 'bilinear', align_corners = False)
        #print(l.size(),g.size())
        c = self.op(l+g) # batch_sizex1xWxH

        a = F.softmax(c.view(N,1,-1), dim=2).view(N,1,W,H)
        g = torch.mul(a.expand_as(l), l)
        g = g.view(N,C,-1).sum(dim=2) # batch_sizexC

        return a, g


In [12]:
class ProjectorBlock(nn.Module):
    def __init__(self, in_features, out_features):

        super(ProjectorBlock, self).__init__()

        self.op = nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=1, padding=0, bias=False)

    def forward(self, inputs):

        return self.op(inputs)

In [13]:
import torchvision.models as models
import torch

class Attn_ResNet34(nn.Module):

    def __init__(self,num_classes):

        super(Attn_ResNet34, self).__init__()

        base_model = models.resnet34(weights = "IMAGENET1K_V1")

        layers = list(base_model.children())

        self.conv1 = nn.Sequential(*layers[0:3])

        self.conv2 = nn.Sequential(*layers[3:5])

        self.conv3 = layers[5]

        self.conv4 = layers[6]

        self.conv5 = layers[7]

        self.drp1 = nn.Dropout(0.3)
        self.drp2 = nn.Dropout(0.4)

        self.pool  = nn.AdaptiveAvgPool2d((1,1))

        self.cls = nn.Linear(in_features = 512*3, out_features = num_classes, bias = True)

        self.projector1 = ProjectorBlock(128,512)
        self.projector2 = ProjectorBlock(256,512)

        self.attn1 = Attn(512,4)
        self.attn2 = Attn(512,2)

    def forward(self,x):

        x = self.conv1(x)
        x = self.conv2(x)
        l1 = self.conv3(x)
        l2 = self.conv4(l1)
        x = self.conv5(l2)

        N, __, __, __ = x.size()
        g = self.pool(x).view(N,512)

        #print(x.size(),g.size(),l1.size(),l2.size())

        a1,g1 = self.attn1(self.projector1(l1),x)
        a2,g2 = self.attn2(self.projector2(l2),x)

        g = torch.cat((g,g1,g2), dim = 1)

        g = self.drp1(g)
        g = self.drp2(g)

        x = self.cls(g)

        x = F.softmax(x,dim = 1)

        return x,a1,a2


In [14]:
x = Attn_ResNet34(3)

In [15]:
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_labels = np.unique(y_train.values)
class_weights = compute_class_weight(class_weight = 'balanced', classes = class_labels, y = y_train.values)
class_weights = torch.from_numpy(class_weights).float()
criterion = nn.CrossEntropyLoss(class_weights)
optimizer = optim.Adam(x.parameters(), lr=0.001)

In [16]:
train_loss = []
val_loss = []
best_loss = 1000

for epoch in range(10):  # loop over the dataset multiple times

    Average_train_loss_per_epoch = 0.0
    Average_val_loss_per_epoch = 0.0

    for i, data in enumerate(train_dataloader, 0):

        running_train_loss_per_batch = 0.0
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs,_,_ = x(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_train_loss_per_batch += loss.item()
        Average_train_loss_per_epoch += running_train_loss_per_batch

        print(f'[{epoch + 1}, {i+1:3d}] loss: {running_train_loss_per_batch}')

    print("\n")
    print(f'epoch[{epoch + 1}] loss: {Average_train_loss_per_epoch / len(train_dataloader)}')
    train_loss.append(Average_train_loss_per_epoch / len(train_dataloader))
    print("\n")

    with torch.no_grad():

        for i, test_data in enumerate(valid_dataloader,0):

            running_val_loss_per_batch = 0.0

            val_inputs,val_labels = test_data

            y_val,_,_ = x(val_inputs)

            loss = criterion(y_val,val_labels)

            running_val_loss_per_batch+=loss.item()
            Average_val_loss_per_epoch+=running_val_loss_per_batch

    val_loss_per_epoch = Average_val_loss_per_epoch / len(valid_dataloader)
    print(f'epoch[{epoch + 1}] val_loss: {val_loss_per_epoch}')
    val_loss.append(val_loss_per_epoch)
    if(val_loss_per_epoch < best_loss):

      torch.save(x.state_dict(),"./rice.h5")
      best_loss = val_loss_per_epoch


print('Finished Training')

[1,   1] loss: 1.114514708518982
[1,   2] loss: 0.9591074585914612
[1,   3] loss: 0.9233496189117432
[1,   4] loss: 0.9070815443992615
[1,   5] loss: 0.9386276006698608
[1,   6] loss: 0.9133873581886292
[1,   7] loss: 0.8661696314811707
[1,   8] loss: 0.8296011686325073
[1,   9] loss: 0.8638337254524231
[1,  10] loss: 0.8666138052940369
[1,  11] loss: 0.8495779037475586
[1,  12] loss: 0.8339512944221497
[1,  13] loss: 0.8634063005447388


epoch[1] loss: 0.9022478552965018


epoch[1] val_loss: 0.8578185737133026
[2,   1] loss: 0.8160419464111328
[2,   2] loss: 0.8405736088752747
[2,   3] loss: 0.8351431488990784
[2,   4] loss: 0.8364018797874451
[2,   5] loss: 0.8166177272796631
[2,   6] loss: 0.79070645570755
[2,   7] loss: 0.8253620862960815
[2,   8] loss: 0.8707500696182251
[2,   9] loss: 0.8409362435340881
[2,  10] loss: 0.789943277835846
[2,  11] loss: 0.8319344520568848
[2,  12] loss: 0.8544171452522278
[2,  13] loss: 0.8549278378486633


epoch[2] loss: 0.831058144569397


epoch[2

In [31]:
!unzip /content/gdrive/MyDrive/test.zip

Archive:  /content/gdrive/MyDrive/test.zip
  inflating: test/id_00vl5wvxq3.jpg  
  inflating: test/id_01hu05mtch.jpg  
  inflating: test/id_030ln10ewn.jpg  
  inflating: test/id_03z57m8xht.jpg  
  inflating: test/id_04ngep1w4b.jpg  
  inflating: test/id_050c6jzczt.jpg  
  inflating: test/id_062s7kr3jp.jpg  
  inflating: test/id_066v0v19bm.jpg  
  inflating: test/id_07v2dwvq17.jpg  
  inflating: test/id_084yoijkju.jpg  
  inflating: test/id_089nmtd1v0.jpg  
  inflating: test/id_0ane7k90m0.jpg  
  inflating: test/id_0e7wk9uzq3.jpg  
  inflating: test/id_0f43lwh2ve.jpg  
  inflating: test/id_0gutmn510g.jpg  
  inflating: test/id_0ivqysjaf3.jpg  
  inflating: test/id_0jv00ddllp.jpg  
  inflating: test/id_0jvgjlu6db.jpg  
  inflating: test/id_0kb1k9p3ox.jpg  
  inflating: test/id_0ptdk3a5vq.jpg  
  inflating: test/id_0r2hdgahag.jpg  
  inflating: test/id_0rye0m9k8n.jpg  
  inflating: test/id_0s71zmqgab.jpg  
  inflating: test/id_0s8jfre56d.jpg  
  inflating: test/id_0s8p3xe7ox.jpg  
  infla

In [32]:
test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [33]:
class Rice_Dataset_Test(Dataset):

    def __init__(self, images, img_dir,transform):

        self.img_dir = img_dir
        self.images = images
        self.transform = transform

    def __len__(self):

        return len(self.images)

    def __getitem__(self,idx):

        img_path = os.path.join(self.img_dir, self.images[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = self.transform(img)
        image_name = self.images[idx]

        return img,image_name


In [34]:
test_df = pd.read_csv("Test.csv")
test_df

,Image_id
0,id_00vl5wvxq3.jpg
1,id_00vl5wvxq3_rgn.jpg
2,id_01hu05mtch.jpg
3,id_01hu05mtch_rgn.jpg
4,id_030ln10ewn.jpg
...,...
2285,id_ztvp2l9k3h_rgn.jpg
2286,id_zwwcma7hlt.jpg
2287,id_zwwcma7hlt_rgn.jpg
2288,id_zyo7m4fj8h.jpg


In [35]:
test_df.drop(axis = 0, index = [i for i in range(1,test_df.shape[0],2)], inplace = True)
test_df.reset_index(inplace = True)
test_df.drop('index',axis = 1, inplace = True)
test_df

,Image_id
0,id_00vl5wvxq3.jpg
1,id_01hu05mtch.jpg
2,id_030ln10ewn.jpg
3,id_03z57m8xht.jpg
4,id_04ngep1w4b.jpg
...,...
1140,id_zrdlgjrq3r.jpg
1141,id_zsfayxwipp.jpg
1142,id_ztvp2l9k3h.jpg
1143,id_zwwcma7hlt.jpg


In [36]:
test_dataset = Rice_Dataset_Test(test_df['Image_id'].values,"/content/test/",transform = test_transform )

In [37]:
test_dataloader = DataLoader(test_dataset, batch_size = 150, shuffle = False)

In [38]:
x = Attn_ResNet34(3)
x.load_state_dict(torch.load(PATH))
x.eval()

Attn_ResNet34(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [39]:
df = {'Image_id':[], 'blast':[], 'brown':[], 'healthy':[]}

with torch.no_grad():

    for data in test_dataloader:

        images,names = data

        outputs,_,_ = x(images)

        _,predicted = torch.max(outputs,1)

        for i in range(len(predicted)):

            df['Image_id'].append(names[i])

            if(predicted[i] == 0):

                df['blast'].append(1)
                df['brown'].append(0)
                df['healthy'].append(0)

            elif(predicted[i] == 1):

                df['blast'].append(0)
                df['brown'].append(1)
                df['healthy'].append(0)

            else:

                df['blast'].append(0)
                df['brown'].append(0)
                df['healthy'].append(1)


submission = pd.DataFrame(df)


In [40]:
submission

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,1,0,0
1,id_01hu05mtch.jpg,0,1,0
2,id_030ln10ewn.jpg,0,0,1
3,id_03z57m8xht.jpg,1,0,0
4,id_04ngep1w4b.jpg,0,0,1
...,...,...,...,...
1140,id_zrdlgjrq3r.jpg,0,0,1
1141,id_zsfayxwipp.jpg,0,1,0
1142,id_ztvp2l9k3h.jpg,0,1,0
1143,id_zwwcma7hlt.jpg,1,0,0


In [41]:
from google.colab import files
submission.to_csv('submission.csv', encoding = 'utf-8-sig')
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>